In [4]:
%config IPCompleter.use_jedi = False
%pdb off
# %load_ext viztracer
# from viztracer import VizTracer
%load_ext autoreload
%autoreload 3
import sys
from pathlib import Path
import os
from datetime import datetime
from typing import Any, List
import pandas as pd
import globus_sdk
from globus_sdk import AccessTokenAuthorizer, TransferClient, TransferData
from globus_sdk.scopes import TransferScopes
from attrs import define, field, Factory

from phoglobushelpers.PhoGlobusHelper import GlobusConnector, KnownEndpoints
from phoglobushelpers.compatibility_objects.Bookmarks import Bookmark, BookmarkList


Automatic pdb calling has been turned OFF
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
connect_man = GlobusConnector.login_and_get_transfer_client()
transfer_client = connect_man.transfer_client
connect_man.list_endpoints()


Please go to this URL and login: https://auth.globus.org/v2/oauth2/authorize?client_id=769d24e1-d1cc-4198-9ff7-2626485da449&redirect_uri=https%3A%2F%2Fauth.globus.org%2Fv2%2Fweb%2Fauth-code&scope=openid+profile+email+urn%3Aglobus%3Aauth%3Ascope%3Atransfer.api.globus.org%3Aall&state=_default&response_type=code&code_challenge=T5CDtDEwDpdEUuxbbp1-JJQDY7c47Hr6h_u7qsb742E&code_challenge_method=S256&access_type=offline
	 Copied url to clipboard!
My Endpoints:
[c3a90494-2555-11ec-a47d-a50ad076c282] aleinbook-linux
[84991054-07b4-11ed-8d83-a54cf61939f8] Apogee
[6d0251c4-2585-11ec-9e35-3df4ed83d858] Cubix Win10
[c3578f36-42a8-11ec-a50f-b537d6c07c1d] FlatEdge_Server
[af3fcfce-f664-11ed-9a7d-83ef71fbf0ae] LNX00052_Fedora
[20c84240-1eb1-11eb-81b7-0e2f230cc907] Pho Personal Laptop
[560d3a12-be20-11ed-9916-cb2cff506ca5] Pho_Personal_Testix
[debb635c-2556-11ec-a47d-a50ad076c282] rMBP Pink Dot
[e414f584-2556-11ec-a0a7-6b21ca6daf73] rMBP Pink Dot


In [16]:
# [Bookmark(bookmark_id='9a8eccc8-2558-11ec-a0a7-6b21ca6daf73', name='aleinbook-linux - PhoPy3DPositionAnalysis', endpoint_id='c3a90494-2555-11ec-a47d-a50ad076c282', path='/~/repo/PhoPy3DPositionAnalysis/'),
#  Bookmark(bookmark_id='2a9f2464-01ad-11ee-be0e-195c41bc0be4', name='Apogee Data W', endpoint_id='84991054-07b4-11ed-8d83-a54cf61939f8', path='/~/W/Data/'),
#  Bookmark(bookmark_id='caf6bf20-9c14-11ed-a29d-8383522b48d9', name='Apogee greatlakes_halechr Folder', endpoint_id='84991054-07b4-11ed-8d83-a54cf61939f8', path='/W/Data/greatlakes_halechr/'),
#  Bookmark(bookmark_id='e1ef1556-4985-11ed-89d5-ede5bae4f491', name='Apogee Personal Dropbox Active', endpoint_id='84991054-07b4-11ed-8d83-a54cf61939f8', path='/E/Dropbox (Personal)/Active/'),
#  Bookmark(bookmark_id='63655524-164e-11ee-bfba-195c41bc0be4', name='Apogee - PersonalFastSwapShare', endpoint_id='84991054-07b4-11ed-8d83-a54cf61939f8', path='/W/FastSwap/PersonalFastSwapShare/'),
#  Bookmark(bookmark_id='1c725da6-203a-11ee-80bc-a3018385fcef', name='Apogee - ProgrammaticDisplayFunctionTesting', endpoint_id='84991054-07b4-11ed-8d83-a54cf61939f8', path='/C/Users/pho/repos/Spike3DWorkEnv/Spike3D/EXTERNAL/Screenshots/ProgrammaticDisplayFunctionTesting/'),
#  Bookmark(bookmark_id='4152f10a-a17b-11ed-a2a4-8383522b48d9', name='Apogee R VMs', endpoint_id='84991054-07b4-11ed-8d83-a54cf61939f8', path='/R/FastSwap/Virtual Machines/'),
#  Bookmark(bookmark_id='78ca952e-2586-11ec-9e35-3df4ed83d858', name='Cubix Win10 PhoPy3DPositionAnalysis2021', endpoint_id='6d0251c4-2585-11ec-9e35-3df4ed83d858', path='/~/repos/PhoPy3DPositionAnalysis2021/'),
#  Bookmark(bookmark_id='e5c8531c-3b77-11ec-adf2-0748e6f5ffe3', name='Cubix Win10 - PhoPy3DPositionAnalysis2021 - data', endpoint_id='6d0251c4-2585-11ec-9e35-3df4ed83d858', path='/~/repos/PhoPy3DPositionAnalysis2021/data/'),
#  Bookmark(bookmark_id='3844ee50-9826-11ed-a849-256017f36728', name='Diba Lab Workstation 2022 - Cloud', endpoint_id='f418ea94-07aa-11ed-8d83-a54cf61939f8', path='/home/halechr/cloud/'),
#  Bookmark(bookmark_id='3995215e-b76a-11ed-a982-5f0e34a3cc4f', name='Diba Lab Workstation 2022 - Cloud - MED-DibaLabDropbox', endpoint_id='f418ea94-07aa-11ed-8d83-a54cf61939f8', path='/home/halechr/cloud/Dropbox_Diba_Shared/MED-DibaLabDropbox/'),
#  Bookmark(bookmark_id='561e4ad0-b76a-11ed-a982-5f0e34a3cc4f', name='Diba Lab Workstation 2022 - Cloud - MED-DibaLabDropbox/Data/', endpoint_id='f418ea94-07aa-11ed-8d83-a54cf61939f8', path='/home/halechr/cloud/Dropbox_Diba_Shared/MED-DibaLabDropbox/Data/'),
#  Bookmark(bookmark_id='ba158112-dfd2-11ed-9b9b-c9bb788c490e', name='Diba Lab Workstation 2022 - GDrive Diba Shared', endpoint_id='f418ea94-07aa-11ed-8d83-a54cf61939f8', path='/media/MAX/cloud/GDrive_Diba_Shared/Data/'),
#  Bookmark(bookmark_id='55246d18-9ce3-11ed-b577-33287ee02ec7', name='Diba Lab Workstation 2022 - greatlakes homedir', endpoint_id='f418ea94-07aa-11ed-8d83-a54cf61939f8', path='/media/MAX/greatlakes_halechr/'),
#  Bookmark(bookmark_id='f7bd6c36-9825-11ed-a849-256017f36728', name='Diba Lab Workstation 2022 - MAX', endpoint_id='f418ea94-07aa-11ed-8d83-a54cf61939f8', path='/media/MAX/Data/'),
#  Bookmark(bookmark_id='431c951e-96c6-11ed-9b93-19370d280681', name='kdiba DataDen Data', endpoint_id='ab65757f-00f5-4e5b-aa21-133187732a01', path='/umms-dibalab/Data/'),
#  Bookmark(bookmark_id='8ce139f8-9d46-11ed-a2a2-8383522b48d9', name='KDIBA Lab Turbo', endpoint_id='8c185a84-5c61-4bbc-b12b-11430e20010f', path='/umms-kdiba/'),
#  Bookmark(bookmark_id='728fb3e8-2597-11ee-80c2-a3018385fcef', name='KDIBA Lab Turbo - Data Global Root Folder', endpoint_id='8c185a84-5c61-4bbc-b12b-11430e20010f', path='/umms-kdiba/Data/'),
#  Bookmark(bookmark_id='248a9a30-164e-11ee-bfba-195c41bc0be4', name='LNX00052_Fedora - MAX', endpoint_id='af3fcfce-f664-11ed-9a7d-83ef71fbf0ae', path='/media/MAX/'),
#  Bookmark(bookmark_id='af68b1ea-20c7-11ee-818f-195c41bc0be4', name='LNX00052_Fedora - ProgrammaticDisplayFunctionTesting', endpoint_id='af3fcfce-f664-11ed-9a7d-83ef71fbf0ae', path='/home/halechr/repo/Spike3D/EXTERNAL/Screenshots/ProgrammaticDisplayFunctionTesting/'),
#  Bookmark(bookmark_id='034e1614-3c19-11ec-90c8-7fa38c35632a', name='NYU Langone Health DTN1 - Buzsakilab Public Datasets', endpoint_id='188a6110-96db-11eb-b7a9-f57b2d55370d', path='/'),
#  Bookmark(bookmark_id='0bc636fc-3997-11ec-9f1c-db52857db67d', name='Pho Personal Laptop - Hiro_Datasets', endpoint_id='20c84240-1eb1-11eb-81b7-0e2f230cc907', path='/Volumes/iNeo/Data/Rotation_3_Kamran Diba Lab/DataProcessingProject/'),
#  Bookmark(bookmark_id='1805df24-e618-11ed-9a63-83ef71fbf0ae', name='Pho Personal Laptop - PegasusR6 - Data', endpoint_id='20c84240-1eb1-11eb-81b7-0e2f230cc907', path='/Volumes/Pegasus R6/Data/'),
#  Bookmark(bookmark_id='bc4aa210-3b77-11ec-adf2-0748e6f5ffe3', name='Pho Personal Laptop - PhoMatlabDataScripting Exported Data', endpoint_id='20c84240-1eb1-11eb-81b7-0e2f230cc907', path='/~/repo/Python Projects/PhoNeuronGillespie2021CodeRepo/PhoMatlabDataScripting/'),
#  Bookmark(bookmark_id='8f77467e-2558-11ec-a0a7-6b21ca6daf73', name='Pho Personal Laptop - PhoPy3DPositionAnalysis', endpoint_id='20c84240-1eb1-11eb-81b7-0e2f230cc907', path='/~/repo/Python Projects/PhoPy3DPositionAnalysis/'),
#  Bookmark(bookmark_id='99ce1e08-bdf1-11ed-8cec-f9fa098153fc', name='UMich Diba Turbo - Pho Directory', endpoint_id='8c185a84-5c61-4bbc-b12b-11430e20010f', path='/umms-kdiba/Pho/'),
#  Bookmark(bookmark_id='b1668f08-2036-11ee-80bc-a3018385fcef', name='umich#greatlakes - halechr homedir', endpoint_id='454f457e-a41b-4807-8775-d132f15a228f', path='/home/halechr/'),
#  Bookmark(bookmark_id='b0569b9c-2558-11ec-a0a7-6b21ca6daf73', name='umich#greatlakes - halechr home folder', endpoint_id='e0370902-9f48-11e9-821b-02b7a92d8e58', path='/~/'),
#  Bookmark(bookmark_id='f479daf4-2039-11ee-80bc-a3018385fcef', name='umich#greatlakes - ProgrammaticDisplayFunctionTesting', endpoint_id='454f457e-a41b-4807-8775-d132f15a228f', path='/home/halechr/repos/Spike3D/EXTERNAL/Screenshots/ProgrammaticDisplayFunctionTesting/'),
#  Bookmark(bookmark_id='bef81650-96c5-11ed-9b93-19370d280681', name='umms-dibalab DataDen', endpoint_id='ab65757f-00f5-4e5b-aa21-133187732a01', path='/umms-dibalab/')]

bookmark_list = connect_man.get_bookmarks()
bookmark_list

[Bookmark(bookmark_id='9a8eccc8-2558-11ec-a0a7-6b21ca6daf73', name='aleinbook-linux - PhoPy3DPositionAnalysis', endpoint_id='c3a90494-2555-11ec-a47d-a50ad076c282', path='/~/repo/PhoPy3DPositionAnalysis/'),
 Bookmark(bookmark_id='2a9f2464-01ad-11ee-be0e-195c41bc0be4', name='Apogee Data W', endpoint_id='84991054-07b4-11ed-8d83-a54cf61939f8', path='/~/W/Data/'),
 Bookmark(bookmark_id='caf6bf20-9c14-11ed-a29d-8383522b48d9', name='Apogee greatlakes_halechr Folder', endpoint_id='84991054-07b4-11ed-8d83-a54cf61939f8', path='/W/Data/greatlakes_halechr/'),
 Bookmark(bookmark_id='e1ef1556-4985-11ed-89d5-ede5bae4f491', name='Apogee Personal Dropbox Active', endpoint_id='84991054-07b4-11ed-8d83-a54cf61939f8', path='/E/Dropbox (Personal)/Active/'),
 Bookmark(bookmark_id='63655524-164e-11ee-bfba-195c41bc0be4', name='Apogee - PersonalFastSwapShare', endpoint_id='84991054-07b4-11ed-8d83-a54cf61939f8', path='/W/FastSwap/PersonalFastSwapShare/'),
 Bookmark(bookmark_id='1c725da6-203a-11ee-80bc-a3018385fc

In [17]:
target_bookmark = Bookmark(bookmark_id='728fb3e8-2597-11ee-80c2-a3018385fcef', name='KDIBA Lab Turbo - Data Global Root Folder', endpoint_id='8c185a84-5c61-4bbc-b12b-11430e20010f', path='/umms-kdiba/Data/')
target_bookmark





Bookmark(bookmark_id='728fb3e8-2597-11ee-80c2-a3018385fcef', name='KDIBA Lab Turbo - Data Global Root Folder', endpoint_id='8c185a84-5c61-4bbc-b12b-11430e20010f', path='/umms-kdiba/Data/')

In [6]:
connect_man.list_files_filtered_by_modified_time(KnownEndpoints.globus_endpoint_kdiba_lab_turbo)

AttributeError: 'TransferClient' object has no attribute 'endpoint_manager_task_submit'

In [23]:
connect_man.list_files(endpoint=target_bookmark.endpoint_id, path=target_bookmark.path, start_date=None, end_date=None)

{
  "DATA": [
    {
      "DATA_TYPE": "file",
      "group": "umms-kdiba-turbo",
      "last_modified": "2023-04-17 23:40:01+00:00",
      "link_group": null,
      "link_last_modified": null,
      "link_size": null,
      "link_target": null,
      "link_user": null,
      "name": "KDIBA",
      "permissions": "2770",
      "size": 106,
      "type": "dir",
      "user": "halechr"
    },
    {
      "DATA_TYPE": "file",
      "group": "umms-kdiba-turbo",
      "last_modified": "2023-04-27 00:06:46+00:00",
      "link_group": null,
      "link_last_modified": null,
      "link_size": null,
      "link_target": null,
      "link_user": null,
      "name": "Output",
      "permissions": "2770",
      "size": 84,
      "type": "dir",
      "user": "halechr"
    },
    {
      "DATA_TYPE": "file",
      "group": "umms-kdiba-turbo",
      "last_modified": "2023-07-14 14:20:27+00:00",
      "link_group": null,
      "link_last_modified": null,
      "link_size": null,
      "link_target": 

IterableTransferResponse({
  "DATA": [
    {
      "DATA_TYPE": "file",
      "group": "umms-kdiba-turbo",
      "last_modified": "2023-04-17 23:40:01+00:00",
      "link_group": null,
      "link_last_modified": null,
      "link_size": null,
      "link_target": null,
      "link_user": null,
      "name": "KDIBA",
      "permissions": "2770",
      "size": 106,
      "type": "dir",
      "user": "halechr"
    },
    {
      "DATA_TYPE": "file",
      "group": "umms-kdiba-turbo",
      "last_modified": "2023-04-27 00:06:46+00:00",
      "link_group": null,
      "link_last_modified": null,
      "link_size": null,
      "link_target": null,
      "link_user": null,
      "name": "Output",
      "permissions": "2770",
      "size": 84,
      "type": "dir",
      "user": "halechr"
    },
    {
      "DATA_TYPE": "file",
      "group": "umms-kdiba-turbo",
      "last_modified": "2023-07-14 14:20:27+00:00",
      "link_group": null,
      "link_last_modified": null,
      "link_size": nu

In [22]:
transfer_client: TransferClient = connect_man.transfer_client

start_date = None
end_date = None

# from bookmark
ep_id = target_bookmark.endpoint_id # '728fb3e8-2597-11ee-80c2-a3018385fcef'
path = target_bookmark.path
start_date = start_date or "2023-07-01" # like "2021-01-01"
end_date = end_date or "" # like "2021-01-01"
response_dict = transfer_client.operation_ls(
    ep_id,
    path=path,
    orderby=["type", "name"],
    filter={"last_modified": [start_date, end_date]},
)

for entry in response_dict:
    print(entry["name"], entry["type"])
    


across_session_result_long_short_inst_firing_rate.pkl file
across_session_result_long_short_inst_firing_rate_2023-07-14.pkl file
global_batch_result_2023-07-07.pkl file
global_batch_result_2023-07-07_extra.pkl file
global_batch_result_2023-07-14.pkl file
long_short_inst_firing_rate_result_handlers.pkl file
long_short_inst_firing_rate_result_handlers_2023-07-12.pkl file
long_short_inst_firing_rates.pkl file
